In [35]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Quatre Cocos,MU,-20.2078,57.7625,71.10,71,38,13.58,scattered clouds
1,1,Lagoa,PT,39.0500,-27.9833,70.70,88,100,6.69,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,51.10,77,0,12.66,clear sky
3,3,Bethel,US,41.3712,-73.4140,53.85,59,100,5.01,overcast clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,46.51,66,0,10.36,clear sky


In [26]:
# 2. Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 98


In [27]:
# 3. Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Quatre Cocos,MU,-20.2078,57.7625,71.10,71,38,13.58,scattered clouds
1,1,Lagoa,PT,39.0500,-27.9833,70.70,88,100,6.69,overcast clouds
9,9,Cabo San Lucas,MX,22.8909,-109.9124,92.61,61,84,6.80,broken clouds
11,11,Mogadishu,SO,2.0371,45.3438,76.68,81,83,16.64,broken clouds
12,12,Atuona,PF,-9.8000,-139.0333,76.98,75,22,17.07,few clouds
14,14,Cayenne,GF,4.9333,-52.3333,82.44,78,0,3.44,clear sky
19,19,Arlit,NE,18.7369,7.3853,93.13,11,0,13.53,clear sky
20,20,Butaritari,KI,3.0707,172.7902,82.35,75,66,16.51,light rain
21,21,Saint George,US,37.1041,-113.5841,85.95,37,0,3.00,clear sky
22,22,Ponta Do Sol,PT,32.6667,-17.1000,70.43,80,24,3.80,few clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()



City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                305
City                   305
Country                305
Lat                    305
Lng                    305
Max Temp               305
Humidity               305
Cloudiness             305
Wind Speed             305
Current Description    305
dtype: int64

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,71.10,scattered clouds,-20.2078,57.7625,
1,Lagoa,PT,70.70,overcast clouds,39.0500,-27.9833,
2,Cabo San Lucas,MX,92.61,broken clouds,22.8909,-109.9124,
3,Mogadishu,SO,76.68,broken clouds,2.0371,45.3438,
4,Atuona,PF,76.98,few clouds,-9.8000,-139.0333,
...,...,...,...,...,...,...,...
300,Velur,IN,75.31,overcast clouds,12.9333,79.1333,
301,Flores,GT,78.85,overcast clouds,16.9226,-89.8994,
302,Grajau,BR,84.34,clear sky,-5.8194,-46.1386,
303,Araouane,ML,93.18,clear sky,18.9048,-3.5265,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}


In [33]:
# Iterate through the hotel DataFrame 
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"


    
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
#     print(hotels["results"][0]["name"])

# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
# Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,71.10,scattered clouds,-20.2078,57.7625,Sunrise Attitude Hotel
1,Lagoa,PT,70.70,overcast clouds,39.0500,-27.9833,Casa Das Faias
2,Cabo San Lucas,MX,92.61,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
3,Mogadishu,SO,76.68,broken clouds,2.0371,45.3438,Hotel Juba
4,Atuona,PF,76.98,few clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
299,Nacala,MZ,71.65,clear sky,-14.5428,40.6728,Nacala Plaza Business Design Hotel
300,Velur,IN,75.31,overcast clouds,12.9333,79.1333,NSR Lodge A/C
301,Flores,GT,78.85,overcast clouds,16.9226,-89.8994,Quinta Maya Hotel
302,Grajau,BR,84.34,clear sky,-5.8194,-46.1386,Hotel em Grajaú-Ma:Hotel Confiança


In [50]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))